In [69]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from IPython.display import display

In [70]:
COMMODITY = 'magnesium'

EDCR_RESULTS_PATH = f'{COMMODITY}/test/predictions/test'
OG_RESULTS_PATH = f'{COMMODITY}/test/results_test.csv'

In [71]:
def extract_info_from_filename(filename):
    parts = filename.split("Rule")
    epsilon = parts[0].split()[-1]
    models_base = parts[1].split("for")
    models = models_base[0].strip()
    base = models_base[1].split("_predictions")[0].strip()
    return base, models, epsilon

In [72]:
def calculate_metrics(df):
    y_true = df['True']
    y_pred = df['Predicted']
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])
    prior = pd.read_csv(OG_RESULTS_PATH, nrows=1)['Prior'].iloc[0]
    return accuracy, precision, recall, f1, prior

In [73]:
def process_files(path):
    all_results = []  
    for filename in os.listdir(path):
        if filename.startswith('Confident') and filename.endswith("_predictions.csv"):
            base, models, epsilon = extract_info_from_filename(filename)
            df = pd.read_csv(os.path.join(path, filename))
            accuracy, precision, recall, f1, prior = calculate_metrics(df)
            row = {
                'Base': base,
                'Models': models,
                'Epsilon': epsilon,
                'Accuracy': accuracy,
                'Precision (0)': precision[0],
                'Recall (0)': recall[0],
                'F1 (0)': f1[0],
                'Precision (1)': precision[1],
                'Recall (1)': recall[1],
                'F1 (1)': f1[1],
                'Prior': prior  
            }
            all_results.append(row)
    
    # Convert the list of results to a DataFrame
    return pd.DataFrame(all_results)

In [74]:
results_df = process_files(EDCR_RESULTS_PATH)

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [75]:
results_df_one = results_df[results_df['Models'] != 'all']
# pd.set_option('display.max_rows', 200)
results_df_one

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
3,CNNA_256_3,CNNA_256_5,0.5,0.455814,0.857143,0.383721,0.530120,0.231884,0.744186,0.353591,0.2
4,CNNA_256_3,CNNA_64_3,0.5,0.358140,0.854167,0.238372,0.372727,0.215569,0.837209,0.342857,0.2
5,CNNA_256_3,CNNA_64_7,0.5,0.330233,0.868421,0.191860,0.314286,0.214689,0.883721,0.345455,0.2
6,CNNA_256_3,CNN_32_7,0.5,0.316279,0.857143,0.174419,0.289855,0.211111,0.883721,0.340807,0.2
7,CNNA_256_3,CNN_64_7,0.5,0.381395,0.854545,0.273256,0.414097,0.218750,0.813953,0.344828,0.2
11,CNNA_256_3,CNNA_256_5,0.6,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
12,CNNA_256_3,CNNA_64_3,0.6,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
13,CNNA_256_3,CNNA_64_7,0.6,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
14,CNNA_256_3,CNN_32_7,0.6,0.790698,0.806763,0.970930,0.881266,0.375000,0.069767,0.117647,0.2
15,CNNA_256_3,CNN_64_7,0.6,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2


### Base Model

In [76]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] != "dumb_spikes") & (results_df['Base'] != "dumb_non_spikes")]
results_df_all

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,CNNA_256_3,all,0.5,0.269767,0.894737,0.098837,0.178010,0.209184,0.953488,0.343096,0.2
8,CNNA_256_3,all,0.6,0.790698,0.806763,0.970930,0.881266,0.375000,0.069767,0.117647,0.2
16,CNNA_256_3,all,0.7,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
24,CNNA_256_3,all,0.8,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
32,CNNA_256_3,all,0.95,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
40,CNNA_256_3,all,0.9,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2


### Dumb Model (Only Predicts Spikes)

In [77]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] == "dumb_spikes")]
results_df_all

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
2,dumb_spikes,all,0.5,0.2,0.0,0.0,0.0,0.2,1.0,0.333333,0.2
10,dumb_spikes,all,0.6,0.2,0.0,0.0,0.0,0.2,1.0,0.333333,0.2
18,dumb_spikes,all,0.7,0.2,0.0,0.0,0.0,0.2,1.0,0.333333,0.2
26,dumb_spikes,all,0.8,0.2,0.0,0.0,0.0,0.2,1.0,0.333333,0.2
34,dumb_spikes,all,0.95,0.2,0.0,0.0,0.0,0.2,1.0,0.333333,0.2
42,dumb_spikes,all,0.9,0.2,0.0,0.0,0.0,0.2,1.0,0.333333,0.2


### Dumb Model (Only Predicts Non-Spikes)

In [78]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] == "dumb_non_spikes")]
results_df_all

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
1,dumb_non_spikes,all,0.5,0.269767,0.894737,0.098837,0.178010,0.209184,0.953488,0.343096,0.2
9,dumb_non_spikes,all,0.6,0.790698,0.806763,0.970930,0.881266,0.375000,0.069767,0.117647,0.2
17,dumb_non_spikes,all,0.7,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
25,dumb_non_spikes,all,0.8,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
33,dumb_non_spikes,all,0.95,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2
41,dumb_non_spikes,all,0.9,0.800000,0.800000,1.000000,0.888889,0.000000,0.000000,0.000000,0.2


In [79]:
og_df = pd.read_csv(OG_RESULTS_PATH)
og_df = og_df.drop('Unnamed: 0', axis=1)
pd.set_option('display.max_rows', 200)
og_df

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,LSTM,256 layers,0.204651,0.545455,0.034884,0.065574,0.186275,0.883721,0.307692,0.2
1,LSTM,128 layers,0.232558,0.684211,0.075581,0.136126,0.188776,0.860465,0.309623,0.2
2,LSTM,64 layers,0.204651,0.545455,0.034884,0.065574,0.186275,0.883721,0.307692,0.2
3,LSTM,32 layers,0.209302,0.571429,0.046512,0.086022,0.184080,0.860465,0.303279,0.2
4,CNN with Attention,"32 filters, kernel size 7",0.325581,0.829268,0.197674,0.319249,0.206897,0.837209,0.331797,0.2
5,CNN with Attention,"32 filters, kernel size 5",0.623256,0.832117,0.662791,0.737864,0.256410,0.465116,0.330579,0.2
6,CNN with Attention,"32 filters, kernel size 3",0.274419,0.785714,0.127907,0.220000,0.197861,0.860465,0.321739,0.2
7,CNN with Attention,"64 filters, kernel size 7",0.330233,0.868421,0.191860,0.314286,0.214689,0.883721,0.345455,0.2
8,CNN with Attention,"64 filters, kernel size 5",0.493023,0.811881,0.476744,0.600733,0.210526,0.558140,0.305732,0.2
9,CNN with Attention,"64 filters, kernel size 3",0.358140,0.854167,0.238372,0.372727,0.215569,0.837209,0.342857,0.2
